In [5]:
#importing necessary libraries (pandas to read CSV, numpy for array manipulations, plotly for intreactive visualisation)
import pandas as pd
import numpy as np
import plotly.express as px

In [6]:
#taking a pre-installed method from sklearn - linear regression
from sklearn.linear_model import LinearRegression

In [8]:
#this will split my dataset into 2 categories: train & test
from sklearn.model_selection import train_test_split

In [10]:
#metrics to evaluate my model
from sklearn import metrics

In [42]:
#read CSV (using pandas - pd)
data = pd.read_csv('Real_estate_valuation_data_set.csv')

In [43]:
#time for correlation heat map, to see how variables are correlated
#maybe house age is tied with distance to MRI? wanna check that
correlation = data.corr() #I put corr function into a new dataframe
correlation #I bring it up, but it's not easy to understand at this point

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
No,1.000000,-0.048658,-0.032808,-0.013573,-0.012699,-0.010110,-0.011059,-0.028587
X1 transaction date,-0.048658,1.000000,0.017549,0.060880,0.009635,0.035058,-0.041082,0.087491
X2 house age,-0.032808,0.017549,1.000000,0.025622,0.049593,0.054420,-0.048520,-0.210567
X3 distance to the nearest MRT station,-0.013573,0.060880,0.025622,1.000000,-0.602519,-0.591067,-0.806317,-0.673613
X4 number of convenience stores,-0.012699,0.009635,0.049593,-0.602519,1.000000,0.444143,0.449099,0.571005
X5 latitude,-0.010110,0.035058,0.054420,-0.591067,0.444143,1.000000,0.412924,0.546307
X6 longitude,-0.011059,-0.041082,-0.048520,-0.806317,0.449099,0.412924,1.000000,0.523287
Y house price of unit area,-0.028587,0.087491,-0.210567,-0.673613,0.571005,0.546307,0.523287,1.000000


In [44]:
#cosmetics
mask = np.zeros_like(correlation, dtype = bool) #
mask[np.triu_indices_from(mask)] = True #
correlation[mask] = np.nan 
correlation.style.background_gradient(cmap='inferno', axis=None, vmin=-1, vmax=1).set_precision(2).highlight_null('White')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning:

this method is deprecated in favour of `Styler.format(precision=..)`



,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
No,nan,nan,nan,nan,nan,nan,nan,nan
X1 transaction date,-0.05,nan,nan,nan,nan,nan,nan,nan
X2 house age,-0.03,0.02,nan,nan,nan,nan,nan,nan
X3 distance to the nearest MRT station,-0.01,0.06,0.03,nan,nan,nan,nan,nan
X4 number of convenience stores,-0.01,0.01,0.05,-0.60,nan,nan,nan,nan
X5 latitude,-0.01,0.04,0.05,-0.59,0.44,nan,nan,nan
X6 longitude,-0.01,-0.04,-0.05,-0.81,0.45,0.41,nan,nan
Y house price of unit area,-0.03,0.09,-0.21,-0.67,0.57,0.55,0.52,nan


In [56]:
#1 would be max
#- is a negative correlation, + number is a positive correlation
#transaction date has little significance, as well as house age
#CONCLUSION: TOP 4 THINS TO TAKE INTO MODEL ARE MRI DISTANCE, # of stores, latitude, longtitude
#However,  latitude/longtide is very hard to analyze, but it deserves attention
#-0.60 is negative correlation between MRI and #of_store, as distance to MRI gets smaller => price per unit gets higher and vice versa
#0.57 is same for stores quantity and price


In [45]:
#using head function to show 1st 5 rows
data.head()

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,1,2012.917,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.917,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833,5.0,390.56840,5,24.97937,121.54245,43.1


In [46]:
#get info on rows quantity + find out if there are NULL values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414 entries, 0 to 413
Data columns (total 8 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   No                                      414 non-null    int64  
 1   X1 transaction date                     414 non-null    float64
 2   X2 house age                            414 non-null    float64
 3   X3 distance to the nearest MRT station  414 non-null    float64
 4   X4 number of convenience stores         414 non-null    int64  
 5   X5 latitude                             414 non-null    float64
 6   X6 longitude                            414 non-null    float64
 7   Y house price of unit area              414 non-null    float64
dtypes: float64(6), int64(2)
memory usage: 26.0 KB


In [47]:
#find out min, mid, max, std, etc. (just for fun) 
#check that dataset is OK: no nulls, no negative values => all OK
data.describe()

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
count,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000
mean,207.500000,2013.148971,17.712560,1083.885689,4.094203,24.969030,121.533361,37.980193
std,119.655756,0.281967,11.392485,1262.109595,2.945562,0.012410,0.015347,13.606488
min,1.000000,2012.667000,0.000000,23.382840,0.000000,24.932070,121.473530,7.600000
25%,104.250000,2012.917000,9.025000,289.324800,1.000000,24.963000,121.528085,27.700000
50%,207.500000,2013.167000,16.100000,492.231300,4.000000,24.971100,121.538630,38.450000
75%,310.750000,2013.417000,28.150000,1454.279000,6.000000,24.977455,121.543305,46.600000
max,414.000000,2013.583000,43.800000,6488.021000,10.000000,25.014590,121.566270,117.500000


In [48]:
#I drop 3 columns because they are hard to interpet
#not currently interested in their effect on the price
#insant head to see if it went ok
data.pop('X1 transaction date')
data.pop('X5 latitude')
data.pop('X6 longitude')
data.head()

,No,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,Y house price of unit area
0,1,32.0,84.87882,10,37.9
1,2,19.5,306.59470,9,42.2
2,3,13.3,561.98450,5,47.3
3,4,13.3,561.98450,5,54.8
4,5,5.0,390.56840,5,43.1


In [49]:
#also checked the last 5, just in case
data.tail()

,No,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,Y house price of unit area
409,410,13.7,4082.01500,0,15.4
410,411,5.6,90.45606,9,50.0
411,412,18.8,390.96960,7,40.6
412,413,8.1,104.81010,5,52.5
413,414,6.5,90.45606,9,63.9


In [50]:
#renaming the columns for easier navigation
data.rename(columns = {'X2 house age':'AGE', 'X3 distance to the nearest MRT station':'MRI', 'X4 number of convenience stores':'#of_stores', 'Y house price of unit area':'price_per_unit'}, inplace = True)
data.head()

,No,AGE,MRI,#of_stores,price_per_unit
0,1,32.0,84.87882,10,37.9
1,2,19.5,306.59470,9,42.2
2,3,13.3,561.98450,5,47.3
3,4,13.3,561.98450,5,54.8
4,5,5.0,390.56840,5,43.1


In [51]:
#visualisation - stage 1
#I'm getting scatter function from px (plotly), choosing x/y axis
#I'm using ols algorithm for my trendline, OrdinaryLeastSquares is basically linear regression
fig = px.scatter(data, x="AGE", y="price_per_unit", trendline ="ols") #create a scatter plot "fig"
fig.show() #show the plot

In [52]:
#visualisation - stage 2
#same function, but want to see other relationships, therefore plot is fig2
fig2 = px.scatter(data, x="MRI", y="price_per_unit", trendline ="ols") #create a scatter plot "fig"
fig2.show() #show the plot

In [53]:
#visualisation - stage 3
#same function, different relationship
fig3 = px.scatter(data, x="#of_stores", y="price_per_unit", trendline ="ols") #create a scatter plot "fig"
fig3.show() #show the plot

In [58]:
#model preparation
#I need to create a data frame of features, that would become the basis for my target prediction
features = ['AGE', 'MRI', '#of_stores'] #I designate certain columns as features
X = data[features] #I take data from specific columns
X.head() #check that it was ok

,AGE,MRI,#of_stores
0,32.0,84.87882,10
1,19.5,306.59470,9
2,13.3,561.98450,5
3,13.3,561.98450,5
4,5.0,390.56840,5


In [59]:
#check that it still has 414 rows and 3 columns for real
print(X.shape)

(414, 3)


In [63]:
#I choose a specific column from my dataframe as Y
Y = data['price_per_unit'] #Y contains data but only when it pertains to price per unit
print(Y.shape)

(414,)


In [64]:
#empty after , because there is only one element


In [65]:
#I split the above datasets each into 2 parts: one that model uses to teach itself and the other one it uses for testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)

In [68]:
#As a rule: 75/25 or 70/30 for dataset split

print(X_train.shape) 
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(310, 3)
(310,)
(104, 3)
(104,)


In [71]:
model = LinearRegression() #use a function, my model is linear regression now
model.fit(X_train, Y_train) #I'm fitting the model which is linear regression to the training data (learn the coefficients)

LinearRegression()

In [72]:
#I want to see intercept (constant) and all other parametrers
print(model.intercept_)
list(zip(features, model.coef_))

42.55604162679223


[('AGE', -0.21567390434515687),
 ('MRI', -0.005286902658830108),
 ('#of_stores', 1.2711055721578268)]

$Price$ = 42.55 - 0.215* $AGE$- 0.005 * $MRI$ + 1.27*$Stores$

In [73]:
#this variable now contains prediction of model (linear regression)
Y_pred = model.predict(X_test)

In [75]:
#find out the mean squared error
print(np.sqrt(metrics.mean_squared_error(Y_test, Y_pred)))

7.834678866670054


In [78]:
#this is another metric for my model (this is how wrong I will be on average in my predictions)

#RESULT OF WORK: I have a model (equation) to calculate price and I know how wrong I can be 
#Unfortunately, error margin is relatively big: price are double digits, error margin of +/- 7 is relatively big
#Possible solution to improve that is to possibly get rid of outliers and try including longtitude + latitude